# AI Agents Capstone Project: Dynamic Blog Writer
## Building a Multi-Agent System Step by Step

**Course**: AI Explorers: Introduction to AI Agents  
**Project Type**: Capstone - 5 Progressive Exercises  
**Learning Objective**: Apply the ReAct framework and agentic workflows to build a complete multi-agent system

---

## 📚 Course Concepts Review

Before starting, let's review the key concepts from [Session 3 Notes](session_course_notes.html):

### The Agent Formula
**🧠 A Brain (LLM) + 🧰 A Set of Tools = 🤖 An AI Agent**

### The ReAct Framework
- **Thought**: The agent reasons about the problem
- **Action**: The agent chooses and uses a tool
- **Observation**: The agent receives results and continues the loop

### Agentic Workflow Pattern: Evaluator-Optimizer
One agent creates a draft, another agent critiques it, creating a loop of continuous improvement.

---

## 🎯 Project Overview

You'll build a **Dynamic Blog Writer** that uses 5 specialized agents working together:

1. **Persona Architect** 🎭 - Creates writer personas and search strategies
2. **Research Analyst** 🔍 - Conducts web research and gathers information  
3. **Content Synthesizer** ✍️ - Writes blog posts based on persona and research
4. **Critic** 🔍 - Evaluates drafts against quality principles
5. **Editor** ✏️ - Provides feedback and manages the iterative improvement process

This demonstrates the **Evaluator-Optimizer** workflow pattern you learned about!

---

## 🚀 Setup Instructions

### Option 1: Google Colab (Recommended)
1. **Open this notebook** in Google Colab
2. **Add API Key Secret**:
   - Click the 🔑 key icon in the left sidebar
   - Add secret: `GEMINI_API_KEY` 
   - Get your key from: https://ai.google.dev/
   - **Enable notebook access** for the secret
3. **Run Setup**: Execute all setup cells in order
4. **Start Coding**: Begin with Exercise 1

### Option 2: Local Environment (VS Code, Cursor, etc.)
1. **Clone/Download** this project to your local machine
2. **Install Dependencies**:
   ```bash
   pip install -r requirements.txt
   ```
3. **Set API Key** (choose one method):
   - **Environment Variable**: `export GEMINI_API_KEY=your_key_here`
   - **VS Code Settings**: Add to your workspace settings
   - **Direct Input**: The notebook will prompt you for the key
4. **Launch Jupyter**:
   ```bash
   jupyter notebook agent_capstone_exercises.ipynb
   ```
5. **Start Coding**: Execute setup cells and begin with Exercise 1

### Troubleshooting
- **API Key Issues**: Ensure your Gemini API key is valid and properly set
- **Package Issues**: Run `pip install --upgrade -r requirements.txt`
- **Import Errors**: Restart your kernel/runtime after installing packages

## Dependencies and Setup

In [ ]:
# Install required packages
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
install_package('google-generativeai')
install_package('beautifulsoup4')
install_package('requests')

print("All packages installed successfully!")

In [ ]:
# Import required libraries
import os
import json
import datetime
import re
import requests
import time
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass
from pathlib import Path
from urllib.parse import quote_plus

import google.generativeai as genai
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

print("Libraries imported successfully!")

In [ ]:
# Configuration - API Key Setup
try:
    # Try Google Colab secrets first
    from google.colab import userdata
    gemini_api_key = userdata.get('GEMINI_API_KEY')
    print("✅ Using Google Colab secrets")
except ImportError:
    # Fallback for local Jupyter
    gemini_api_key = os.environ.get('GEMINI_API_KEY')
    if not gemini_api_key:
        gemini_api_key = input("Enter your Gemini API key: ")
    print("✅ Using local environment/input")

# Configure Gemini
if not gemini_api_key:
    raise ValueError("❌ GEMINI_API_KEY not found. Please add it to Colab secrets or environment variables.")

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel('gemini-2.5-pro')

print("🤖 Gemini Pro 2.5 model initialized successfully!")
print("📝 Ready to start building your multi-agent system!")

## Data Classes and Quality Principles

First, let's define the data structures and quality principles that our agents will use.

In [ ]:
# Data classes for our multi-agent system
@dataclass
class PersonaResult:
    persona_prompt: str
    search_queries: List[str]

@dataclass
class SearchResult:
    title: str
    url: str
    snippet: str

@dataclass
class ResearchResult:
    content: str
    source_count: int = 0

@dataclass
class BlogDraft:
    content: str
    version: int

@dataclass
class QualityReview:
    is_approved: bool
    feedback: str
    issues_found: List[str]

@dataclass
class EditorReview:
    is_approved: bool
    comments: str

print("📋 Data classes defined successfully!")

In [ ]:
# Quality Principles - The foundation of good writing
QUALITY_PRINCIPLES = """
FUNDAMENTAL PRINCIPLES OF QUALITY WRITING:

P1: Evidentiary Support
All claims must be directly traceable to the provided research material.

P2: Clarity and Conciseness
Writing must be precise, unambiguous, and free of unnecessary jargon.

P3: Engaging Narrative
The post must feature a strong hook, a logical flow, and a memorable conclusion.

P4: Structural Integrity
The output must be well-organized with a clear title, introduction, body, and conclusion.

P5: Intellectual Honesty
Information must be represented accurately, even when adopting a specific persona.
"""

print("🎯 Quality principles established:")
print(QUALITY_PRINCIPLES)

## Utility Classes

Let's set up some utility classes for web searching and file management.

In [ ]:
class FreeWebSearcher:
    """Simple web search using DuckDuckGo (no API key required)."""
    
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
    
    def search(self, query: str, num_results: int = 5) -> List[SearchResult]:
        """Perform a free web search using DuckDuckGo."""
        try:
            search_url = f"https://html.duckduckgo.com/html/?q={quote_plus(query)}"
            response = self.session.get(search_url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            results = []
            
            for result_div in soup.find_all('div', class_='result')[:num_results]:
                try:
                    title_elem = result_div.find('a', class_='result__a')
                    snippet_elem = result_div.find('a', class_='result__snippet')
                    
                    if title_elem and snippet_elem:
                        title = title_elem.get_text(strip=True)
                        url = title_elem.get('href', '')
                        snippet = snippet_elem.get_text(strip=True)
                        
                        if title and url and snippet:
                            results.append(SearchResult(title=title, url=url, snippet=snippet))
                except Exception:
                    continue
            
            return results
        except Exception as e:
            print(f"Search error: {e}")
            return []

class FileManager:
    """Manages file operations for the blog writing project."""
    
    def __init__(self, topic: str):
        self.topic = topic
        self.date_str = datetime.datetime.now().strftime("%Y%m%d")
        # Create simple folder name
        topic_abbrev = re.sub(r'[^a-zA-Z0-9]', '_', topic.lower())[:20]
        self.folder_name = f"{self.date_str}_{topic_abbrev}"
        self.output_dir = Path(self.folder_name)
        self.output_dir.mkdir(exist_ok=True)
        print(f"📁 Created output directory: {self.output_dir}")
    
    def save_file(self, content: str, filename: str) -> Path:
        filepath = self.output_dir / filename
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"💾 Saved: {filepath}")
        return filepath

# Initialize web searcher
web_searcher = FreeWebSearcher()
print("🔍 Web searcher initialized (using free DuckDuckGo search)")
print("📂 File manager ready")

---

# 🎯 Exercise 1: Persona Architect Agent

## Learning Objectives
- Understand how to create specialized agent personas
- Learn to generate targeted search queries
- Practice JSON output formatting for agent communication

## Background
The **Persona Architect** is your first agent. Its job is to:
1. Analyze the topic and style requirements
2. Create a detailed writer persona 
3. Generate search queries for research

This follows the **ReAct** pattern:
- **Thought**: "I need to create a persona suitable for this topic"
- **Action**: Generate persona and queries using the LLM
- **Observation**: Return structured results for the next agent

## Your Task
Complete the `PersonaArchitect` class by implementing the `generate_persona_and_queries` method.

In [ ]:
class PersonaArchitect:
    """Agent that creates writer personas and search strategies."""
    
    def __init__(self, model):
        self.model = model
    
    def generate_persona_and_queries(self, topic: str, style_and_background: str) -> PersonaResult:
        """Generate a writer persona and search queries for the given topic.
        
        Args:
            topic: The blog post topic
            style_and_background: Style guide and background requirements
            
        Returns:
            PersonaResult with persona_prompt and search_queries
        """
        
        # TODO: Create a prompt that will generate both a writer persona and search queries
        # HINT: The persona should include writing style, expertise, and perspective
        # HINT: Search queries should be diverse and targeted to find current information
        # HINT: Return the result as JSON with "persona_prompt" and "search_queries" fields
        
        prompt = f"""
        # TODO: Write your prompt here
        # Remember to:
        # 1. Ask for a detailed writer persona based on the topic and style
        # 2. Ask for at least 3 search queries
        # 3. Request JSON output format
        # 4. Be specific about what makes a good persona (expertise, style, voice)
        
        TOPIC: {topic}
        STYLE REQUIREMENTS: {style_and_background}
        
        # Your prompt goes here...
        """
        
        # TODO: Make the API call and parse the JSON response
        # HINT: Use self.model.generate_content(prompt)
        # HINT: Clean the response text and use json.loads()
        # HINT: Handle potential JSON parsing errors
        
        try:
            # TODO: Generate content and parse JSON
            response = None  # Replace with actual API call
            
            # TODO: Extract and clean the response text
            response_text = ""  # Replace with cleaned response
            
            # TODO: Parse JSON and create PersonaResult
            data = {}  # Replace with parsed JSON
            
            return PersonaResult(
                persona_prompt="",  # TODO: Extract from data
                search_queries=[]   # TODO: Extract from data
            )
            
        except Exception as e:
            print(f"❌ Error in PersonaArchitect: {e}")
            raise

# Test your implementation
print("🎭 PersonaArchitect class defined")
print("⚠️  TODO: Implement the generate_persona_and_queries method")

# Assertion to check if method is implemented
persona_architect = PersonaArchitect(model)

# This will fail until you implement the method properly
try:
    test_result = persona_architect.generate_persona_and_queries(
        "Artificial Intelligence in Healthcare", 
        "Professional, accessible tone for general audience"
    )
    assert isinstance(test_result.persona_prompt, str) and len(test_result.persona_prompt) > 50
    assert isinstance(test_result.search_queries, list) and len(test_result.search_queries) >= 3
    print("✅ Exercise 1 completed successfully!")
    print(f"📝 Generated persona: {test_result.persona_prompt[:100]}...")
    print(f"🔍 Generated {len(test_result.search_queries)} search queries")
except Exception as e:
    print(f"❌ Exercise 1 incomplete: {e}")
    print("💡 Hint: Make sure to implement the method and return valid PersonaResult")

---

# 🔍 Exercise 2: Research Analyst Agent

## Learning Objectives
- Learn to integrate external tools (web search) with LLMs
- Practice data synthesis and consolidation
- Understand how agents use tools to gather information

## Background
The **Research Analyst** uses the search queries from Exercise 1 to:
1. Perform actual web searches
2. Collect and analyze search results
3. Synthesize findings into coherent research content

This demonstrates how agents use **tools** (the search function) to interact with the world.

## Your Task
Complete the `ResearchAnalyst` class by implementing the research synthesis logic.

In [ ]:
class ResearchAnalyst:
    """Agent that conducts web research and synthesizes findings."""
    
    def __init__(self, model, web_searcher):
        self.model = model
        self.web_searcher = web_searcher
    
    def conduct_research(self, search_queries: List[str]) -> ResearchResult:
        """Conduct web research and synthesize findings.
        
        Args:
            search_queries: List of search queries to execute
            
        Returns:
            ResearchResult with synthesized content and source count
        """
        
        print("🔍 Starting web research...")
        all_results = []
        
        # TODO: Execute each search query and collect results
        # HINT: Use self.web_searcher.search(query) for each query
        # HINT: Combine all results into a single list
        
        for i, query in enumerate(search_queries, 1):
            print(f"   Searching: '{query}'")
            # TODO: Perform the search and add results to all_results
            results = []  # Replace with actual search call
            all_results.extend(results)
            
            # Be respectful to the search service
            if i < len(search_queries):
                time.sleep(1)
        
        print(f"📊 Found {len(all_results)} total search results")
        
        if not all_results:
            return ResearchResult(
                content="--- RESEARCH START ---\nNo search results found.\n--- RESEARCH END ---",
                source_count=0
            )
        
        # TODO: Format search results for LLM analysis
        # HINT: Create a structured text with all search results
        # HINT: Include title, URL, and snippet for each result
        
        search_content = ""
        for i, result in enumerate(all_results, 1):
            # TODO: Format each result
            search_content += ""  # Add formatted result info
        
        # TODO: Create a prompt to analyze and synthesize the search results
        # HINT: Ask the LLM to create a comprehensive research summary
        # HINT: Ask for the output to be wrapped in --- RESEARCH START/END --- markers
        
        analysis_prompt = f"""
        # TODO: Write your analysis prompt here
        # The prompt should:
        # 1. Ask for analysis of the search results
        # 2. Request synthesis into coherent research content
        # 3. Ask for --- RESEARCH START/END --- markers
        # 4. Emphasize accuracy and fact-based synthesis
        
        SEARCH RESULTS TO ANALYZE:
        {search_content}
        
        # Your analysis prompt goes here...
        """
        
        try:
            # TODO: Generate content analysis
            response = None  # Replace with actual API call
            content = ""  # Replace with response text
            
            return ResearchResult(
                content=content,
                source_count=len(all_results)
            )
            
        except Exception as e:
            print(f"❌ Error analyzing search results: {e}")
            # Return basic compilation as fallback
            basic_content = f"--- RESEARCH START ---\nFound {len(all_results)} sources:\n"
            for result in all_results[:5]:  # Top 5
                basic_content += f"- {result.title}: {result.snippet}\n"
            basic_content += "--- RESEARCH END ---"
            
            return ResearchResult(content=basic_content, source_count=len(all_results))

# Test your implementation
print("🔍 ResearchAnalyst class defined")
print("⚠️  TODO: Implement the conduct_research method")

# Test with the PersonaArchitect from Exercise 1
research_analyst = ResearchAnalyst(model, web_searcher)

# This will fail until both Exercise 1 and 2 are implemented
try:
    # Test with sample queries
    test_queries = ["AI healthcare applications 2024", "machine learning medical diagnosis"]
    test_research = research_analyst.conduct_research(test_queries)
    
    assert isinstance(test_research.content, str) and len(test_research.content) > 100
    assert "--- RESEARCH START ---" in test_research.content
    assert "--- RESEARCH END ---" in test_research.content
    assert test_research.source_count >= 0
    
    print("✅ Exercise 2 completed successfully!")
    print(f"📊 Research content length: {len(test_research.content)} characters")
    print(f"🔗 Sources found: {test_research.source_count}")
    
except Exception as e:
    print(f"❌ Exercise 2 incomplete: {e}")
    print("💡 Hint: Make sure to implement web search execution and content synthesis")

---

# ✍️ Exercise 3: Content Synthesizer Agent

## Learning Objectives
- Learn to combine multiple inputs (persona + research) for content generation
- Practice adherence to style guidelines and quality principles
- Understand how to handle iterative improvement (revisions)

## Background
The **Content Synthesizer** is the creative heart of our system. It:
1. Takes the persona and research from previous agents
2. Writes blog posts that follow the quality principles
3. Can revise content based on feedback (iterative improvement)

This demonstrates how agents can work with complex, multi-part inputs.

## Your Task
Complete the `ContentSynthesizer` class by implementing the blog writing logic.

In [ ]:
class ContentSynthesizer:
    """Agent that writes blog posts based on persona and research."""
    
    def __init__(self, model):
        self.model = model
    
    def write_blog_post(self, persona_prompt: str, research_content: str, 
                       topic: str, style_requirements: str, 
                       editorial_feedback: Optional[str] = None) -> BlogDraft:
        """Write a blog post using persona and research.
        
        Args:
            persona_prompt: The writer persona to adopt
            research_content: Synthesized research content
            topic: The blog post topic
            style_requirements: Style and background requirements
            editorial_feedback: Optional feedback for revision
            
        Returns:
            BlogDraft with content and version number
        """
        
        # TODO: Create a comprehensive prompt for blog writing
        # HINT: Include all the input information in your prompt
        # HINT: Reference the QUALITY_PRINCIPLES
        # HINT: Handle the optional editorial_feedback for revisions
        
        base_prompt = f"""
        # TODO: Write your blog writing prompt here
        # The prompt should:
        # 1. Establish the writer persona
        # 2. Provide the research content to work with
        # 3. Reference the quality principles (use QUALITY_PRINCIPLES variable)
        # 4. Specify the topic and style requirements
        # 5. Ask for approximately 500 words
        # 6. Request Markdown format with title
        
        WRITER PERSONA:
        {persona_prompt}
        
        RESEARCH CONTENT:
        {research_content}
        
        TOPIC: {topic}
        
        STYLE REQUIREMENTS:
        {style_requirements}
        
        QUALITY PRINCIPLES TO FOLLOW:
        {QUALITY_PRINCIPLES}
        
        # Your writing instructions go here...
        """
        
        # TODO: Handle editorial feedback for revisions
        # HINT: If editorial_feedback is provided, add it to the prompt
        # HINT: Ask the agent to specifically address the feedback
        
        if editorial_feedback:
            base_prompt += f"""
            
            EDITORIAL FEEDBACK TO ADDRESS:
            {editorial_feedback}
            
            # TODO: Add instructions for handling feedback
            """
        
        try:
            # TODO: Generate the blog post content
            response = None  # Replace with actual API call
            content = ""  # Replace with response text
            
            # TODO: Determine version number (1 for new, 2+ for revisions)
            version = 1  # Replace with logic based on editorial_feedback
            
            return BlogDraft(content=content, version=version)
            
        except Exception as e:
            print(f"❌ Error in ContentSynthesizer: {e}")
            raise

# Test your implementation
print("✍️ ContentSynthesizer class defined")
print("⚠️  TODO: Implement the write_blog_post method")

# Test with sample data
content_synthesizer = ContentSynthesizer(model)

# This will fail until you implement the method
try:
    sample_persona = "Expert technology writer with 10 years of experience in AI and healthcare."
    sample_research = "--- RESEARCH START ---\nAI is transforming healthcare through improved diagnostics and personalized treatment.\n--- RESEARCH END ---"
    
    test_draft = content_synthesizer.write_blog_post(
        persona_prompt=sample_persona,
        research_content=sample_research,
        topic="AI in Healthcare",
        style_requirements="Professional, accessible tone"
    )
    
    assert isinstance(test_draft.content, str) and len(test_draft.content) > 200
    assert test_draft.version == 1
    assert "#" in test_draft.content  # Should have Markdown title
    
    print("✅ Exercise 3 completed successfully!")
    print(f"📝 Blog post length: {len(test_draft.content)} characters")
    print(f"📄 Version: {test_draft.version}")
    
except Exception as e:
    print(f"❌ Exercise 3 incomplete: {e}")
    print("💡 Hint: Make sure to implement blog writing logic with all required inputs")

---

# 🔍 Exercise 4: Critic Agent

## Learning Objectives
- Learn to evaluate content against specific criteria
- Practice providing structured feedback
- Understand quality assessment in AI systems

## Background
The **Critic Agent** evaluates blog drafts against our quality principles. It:
1. Analyzes the draft for adherence to quality principles (P1-P5)
2. Checks factual consistency with research
3. Provides structured feedback with specific issues

This is the "evaluator" part of the **Evaluator-Optimizer** pattern you learned about.

## Your Task
Complete the `CriticAgent` class by implementing the quality evaluation logic.

In [ ]:
class CriticAgent:
    """Agent that evaluates blog drafts against quality principles."""
    
    def __init__(self, model):
        self.model = model
    
    def evaluate_draft(self, draft_content: str, research_content: str, 
                      topic: str) -> QualityReview:
        """Evaluate a blog draft against quality principles.
        
        Args:
            draft_content: The blog draft to evaluate
            research_content: Original research content for fact-checking
            topic: The blog post topic
            
        Returns:
            QualityReview with approval status, feedback, and issues
        """
        
        # TODO: Create a comprehensive evaluation prompt
        # HINT: Ask the LLM to check each quality principle (P1-P5)
        # HINT: Request specific feedback about issues found
        # HINT: Ask for JSON output with approval status and feedback
        
        evaluation_prompt = f"""
        # TODO: Write your evaluation prompt here
        # The prompt should:
        # 1. Present the draft to be evaluated
        # 2. Provide the research content for fact-checking
        # 3. List the quality principles to check against
        # 4. Ask for specific analysis of each principle
        # 5. Request JSON output with approval and detailed feedback
        
        BLOG DRAFT TO EVALUATE:
        {draft_content}
        
        RESEARCH CONTENT FOR FACT-CHECKING:
        {research_content}
        
        TOPIC: {topic}
        
        QUALITY PRINCIPLES TO EVALUATE:
        {QUALITY_PRINCIPLES}
        
        # Your evaluation instructions go here...
        # Remember to ask for JSON format like:
        # {{
        #   "is_approved": true/false,
        #   "feedback": "detailed feedback text",
        #   "issues_found": ["list", "of", "specific", "issues"]
        # }}
        """
        
        try:
            # TODO: Generate the evaluation
            response = None  # Replace with actual API call
            response_text = ""  # Replace with cleaned response
            
            # TODO: Clean and parse JSON response
            # HINT: Remove ```json``` markers if present
            # HINT: Use json.loads() to parse
            
            data = {}  # Replace with parsed JSON
            
            return QualityReview(
                is_approved=False,  # TODO: Extract from data
                feedback="",        # TODO: Extract from data
                issues_found=[]     # TODO: Extract from data
            )
            
        except Exception as e:
            print(f"❌ Error in CriticAgent: {e}")
            # Return safe fallback
            return QualityReview(
                is_approved=False,
                feedback=f"Error during evaluation: {e}",
                issues_found=["Evaluation error occurred"]
            )

# Test your implementation
print("🔍 CriticAgent class defined")
print("⚠️  TODO: Implement the evaluate_draft method")

# Test with sample data
critic_agent = CriticAgent(model)

# This will fail until you implement the method
try:
    sample_draft = "# AI in Healthcare\n\nArtificial Intelligence is transforming healthcare in many ways..."
    sample_research = "--- RESEARCH START ---\nAI applications include medical imaging and drug discovery.\n--- RESEARCH END ---"
    
    test_review = critic_agent.evaluate_draft(
        draft_content=sample_draft,
        research_content=sample_research,
        topic="AI in Healthcare"
    )
    
    assert isinstance(test_review.is_approved, bool)
    assert isinstance(test_review.feedback, str) and len(test_review.feedback) > 10
    assert isinstance(test_review.issues_found, list)
    
    print("✅ Exercise 4 completed successfully!")
    print(f"✅ Approved: {test_review.is_approved}")
    print(f"📝 Feedback length: {len(test_review.feedback)} characters")
    print(f"⚠️  Issues found: {len(test_review.issues_found)}")
    
except Exception as e:
    print(f"❌ Exercise 4 incomplete: {e}")
    print("💡 Hint: Make sure to implement evaluation logic with JSON output parsing")

---

# ✏️ Exercise 5: Editor Agent & Workflow Orchestration

## Learning Objectives
- Learn to orchestrate multiple agents in a workflow
- Implement iterative improvement loops
- Understand the complete **Evaluator-Optimizer** pattern

## Background
The **Editor Agent** manages the iterative improvement process. It:
1. Coordinates between the Content Synthesizer and Critic
2. Manages the revision loop (up to 3 cycles)
3. Makes final approval decisions

This demonstrates the complete **Evaluator-Optimizer** workflow from your course!

## Your Task
Complete the `EditorAgent` class and implement the main workflow orchestration.

In [ ]:
class EditorAgent:
    """Agent that manages the iterative improvement workflow."""
    
    def __init__(self, model):
        self.model = model
    
    def make_editorial_decision(self, quality_review: QualityReview, 
                               current_cycle: int, max_cycles: int) -> EditorReview:
        """Make an editorial decision based on critic feedback.
        
        Args:
            quality_review: Review from the Critic Agent
            current_cycle: Current revision cycle number
            max_cycles: Maximum allowed cycles
            
        Returns:
            EditorReview with final decision and comments
        """
        
        # TODO: Implement editorial decision logic
        # HINT: Consider the quality review, current cycle, and max cycles
        # HINT: Approve if critic approved, or if max cycles reached
        # HINT: Provide constructive feedback for improvements
        
        if quality_review.is_approved:
            # TODO: Handle approval case
            return EditorReview(
                is_approved=True,
                comments=""  # TODO: Create approval message
            )
        elif current_cycle >= max_cycles:
            # TODO: Handle max cycles reached
            return EditorReview(
                is_approved=True,  # Approve despite issues
                comments=""  # TODO: Create max cycles message
            )
        else:
            # TODO: Handle revision needed case
            return EditorReview(
                is_approved=False,
                comments=""  # TODO: Format feedback for revision
            )

class BlogWorkflowOrchestrator:
    """Main orchestrator that coordinates all agents in the workflow."""
    
    def __init__(self, model, web_searcher):
        # TODO: Initialize all agents
        # HINT: Create instances of all the agent classes you built
        self.persona_architect = None    # TODO: Initialize PersonaArchitect
        self.research_analyst = None     # TODO: Initialize ResearchAnalyst
        self.content_synthesizer = None  # TODO: Initialize ContentSynthesizer
        self.critic_agent = None         # TODO: Initialize CriticAgent
        self.editor_agent = None         # TODO: Initialize EditorAgent
    
    def generate_blog_post(self, topic: str, style_and_background: str) -> Tuple[str, Path]:
        """Execute the complete blog generation workflow.
        
        Args:
            topic: Blog post topic
            style_and_background: Style requirements
            
        Returns:
            Tuple of (final_content, final_file_path)
        """
        
        print(f"🚀 Starting blog generation for: '{topic}'")
        
        # Initialize file manager
        file_manager = FileManager(topic)
        
        try:
            # Phase 1: Generate Persona and Queries
            print("\n🎭 Phase 1: Generating writer persona and search queries...")
            # TODO: Use PersonaArchitect to generate persona and queries
            persona_result = None  # Replace with actual call
            
            print(f"   Generated persona with {len(persona_result.search_queries)} search queries")
            
            # Phase 2: Conduct Research
            print("\n🔍 Phase 2: Conducting web research...")
            # TODO: Use ResearchAnalyst to conduct research
            research_result = None  # Replace with actual call
            
            print(f"   Research completed with {research_result.source_count} sources")
            
            # Phase 3: Iterative Content Generation and Review
            print("\n✍️ Phase 3: Starting iterative content creation...")
            
            current_draft = None
            max_cycles = 3
            
            for cycle in range(1, max_cycles + 1):
                print(f"\n   📝 Cycle {cycle}/{max_cycles}")
                
                # Generate or revise draft
                if current_draft is None:
                    print("      Writing initial draft...")
                    # TODO: Use ContentSynthesizer to write initial draft
                    draft = None  # Replace with actual call
                else:
                    print("      Revising draft based on feedback...")
                    # TODO: Use ContentSynthesizer to revise with feedback
                    draft = None  # Replace with actual call
                
                current_draft = draft
                
                # Save draft
                file_manager.save_file(draft.content, f"draft_{cycle}.md")
                
                # Evaluate with Critic
                print("      Evaluating with Critic Agent...")
                # TODO: Use CriticAgent to evaluate the draft
                quality_review = None  # Replace with actual call
                
                # Make editorial decision
                print("      Making editorial decision...")
                # TODO: Use EditorAgent to make decision
                editorial_review = None  # Replace with actual call
                
                # Save review
                file_manager.save_file(
                    f"Approved: {editorial_review.is_approved}\n\n{editorial_review.comments}",
                    f"review_{cycle}.md"
                )
                
                print(f"      Result: {'✅ Approved' if editorial_review.is_approved else '🔄 Needs revision'}")
                
                if editorial_review.is_approved:
                    print(f"   🎉 Blog post approved after {cycle} cycle(s)!")
                    break
                
                # Store feedback for next revision
                last_feedback = editorial_review.comments
            
            # Phase 4: Finalization
            print("\n📄 Phase 4: Finalizing blog post...")
            final_path = file_manager.save_file(current_draft.content, "final_blog.md")
            
            # Display summary
            word_count = len(current_draft.content.split())
            print(f"\n📊 Generation Summary:")
            print(f"   📂 Output folder: {file_manager.folder_name}")
            print(f"   📝 Final word count: {word_count} words")
            print(f"   🔍 Sources used: {research_result.source_count}")
            print(f"   🔄 Revision cycles: {cycle}")
            
            return current_draft.content, final_path
            
        except Exception as e:
            print(f"❌ Error during blog generation: {e}")
            raise

# Test your implementation
print("✏️ EditorAgent and BlogWorkflowOrchestrator classes defined")
print("⚠️  TODO: Complete all agent implementations and orchestration logic")

# This will fail until all exercises are completed
try:
    orchestrator = BlogWorkflowOrchestrator(model, web_searcher)
    
    # Verify all agents are initialized
    assert orchestrator.persona_architect is not None
    assert orchestrator.research_analyst is not None
    assert orchestrator.content_synthesizer is not None
    assert orchestrator.critic_agent is not None
    assert orchestrator.editor_agent is not None
    
    print("✅ Exercise 5 setup completed successfully!")
    print("🎉 All agents initialized and ready for workflow execution")
    
except Exception as e:
    print(f"❌ Exercise 5 incomplete: {e}")
    print("💡 Hint: Make sure all previous exercises are completed and all agents are initialized")

---

# 🎯 Final Integration Test

Once you've completed all 5 exercises, run this cell to test the complete workflow!

In [ ]:
# Final integration test - run the complete workflow
def run_complete_workflow():
    """Run the complete blog generation workflow."""
    
    # Sample inputs
    topic = "The Future of Artificial Intelligence in Healthcare"
    style_and_background = """
    Target audience: General tech-savvy readers interested in healthcare innovation
    Tone: Professional but accessible, optimistic yet balanced
    Style: Informative with real-world examples, approximately 500 words
    Background: Write from the perspective of someone knowledgeable about both AI and healthcare trends
    """
    
    try:
        # Initialize orchestrator
        orchestrator = BlogWorkflowOrchestrator(model, web_searcher)
        
        # Run complete workflow
        final_content, final_path = orchestrator.generate_blog_post(topic, style_and_background)
        
        # Display final result
        print("\n" + "="*70)
        print("🎉 CAPSTONE PROJECT COMPLETED SUCCESSFULLY!")
        print("="*70)
        
        # Show preview of final blog
        preview = final_content[:500] + "..." if len(final_content) > 500 else final_content
        display(Markdown(f"### Final Blog Post Preview\n\n{preview}"))
        
        print(f"\n📄 Full blog saved to: {final_path}")
        print("\n🏆 Congratulations! You've successfully built a multi-agent system!")
        
        return True
        
    except Exception as e:
        print(f"❌ Integration test failed: {e}")
        print("\n🔧 Please review and complete all exercises above.")
        return False

# Run the test when all exercises are complete
print("🧪 Ready to run integration test...")
print("⚠️  Make sure all exercises above are completed first!")

# Uncomment the next line when you're ready to test:
# success = run_complete_workflow()

---

# 🎓 Capstone Project Summary

## What You've Built

Congratulations! You've just built a complete **multi-agent system** that demonstrates all the key concepts from your AI Agents course:

### 🧠 Agent Fundamentals Applied
- **Brain (LLM)**: Each agent uses Gemini Pro 2.5 for reasoning
- **Tools**: Web search, file management, content generation
- **Result**: 5 specialized agents working together

### 🔄 ReAct Framework in Action
- **Thought**: Each agent reasons about its specific task
- **Action**: Agents use tools (search, generate, evaluate)
- **Observation**: Results feed into the next agent or iteration

### 🏭 Evaluator-Optimizer Workflow
- **Creator**: Content Synthesizer writes drafts
- **Evaluator**: Critic Agent reviews against quality principles
- **Optimizer**: Editor Agent manages iterative improvement
- **Loop**: Up to 3 cycles of continuous improvement

## Technical Achievements

✅ **Exercise 1**: Persona generation with targeted search queries  
✅ **Exercise 2**: Real web search integration and research synthesis  
✅ **Exercise 3**: Content generation with persona adherence  
✅ **Exercise 4**: Quality evaluation against structured principles  
✅ **Exercise 5**: Complete workflow orchestration with iterative improvement

## Real-World Applications

The patterns you've learned can be applied to:
- **Content Creation**: Automated writing with quality control
- **Research & Analysis**: Multi-step investigation workflows
- **Code Review**: Automated code evaluation and improvement
- **Decision Support**: Multi-agent consultation systems
- **Creative Projects**: AI collaboration in design and writing

## Next Steps

Now that you understand multi-agent systems, consider exploring:
- **Advanced Workflows**: Try parallel agent execution
- **Specialized Tools**: Add more sophisticated tools to your agents
- **Domain Expertise**: Create agents for specific industries
- **Interactive Systems**: Build agents that can collaborate with humans

---

## 📚 Course Connection

This capstone perfectly demonstrates the progression from your course:

1. **Session 1**: Prompt Engineering → Used in every agent interaction
2. **Session 2**: RAG → Research integration and knowledge synthesis  
3. **Session 3**: AI Agents → Complete multi-agent system with tools

You've now experienced the full power of giving AI not just knowledge, but the ability to **act**!

---

🎉 **Congratulations on completing your AI Agents Capstone Project!** 🎉